<a href="https://colab.research.google.com/github/RubenRomeroTorrado/gym/blob/main/appedafoclimatica5(automatica).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install joblib gradio folium ipyleaflet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.7 MB/s eta 0:00:00


In [ ]:
import folium
import gradio as gr
import joblib
import json
import numpy as np
from google.colab import drive
from folium.plugins import MarkerCluster
from IPython.display import Javascript

In [ ]:
# Cargar modelo y scaler
modelo = joblib.load("/content/modelo_random_forest_tempesolo5.pkl")
scaler = joblib.load("/content/scaler5.pkl")

In [ ]:
from ipyleaflet import Map, Marker
from ipywidgets import Output, Button, VBox
from IPython.display import display
from google.colab import output
import pandas as pd

# Habilitar el manejador de widgets personalizados
output.enable_custom_widget_manager()

# Configuración inicial del mapa
m = Map(center=(39.5, -8), zoom=7)
coord_output = Output()
current_marker = None

# DataFrame para guardar resultados
resultados_df = pd.DataFrame(columns=["Latitud", "Longitud"] + [
    "Media_Dias_geada_anual",
    "Media_de_Horas_de_Frio",
    "Media_de_Precipitação_ERA5",
    "Minimo_de_Media_de_Temperatura_Minima(ºC)",
    "Média de Média de Temperatura Mínima (ºC)",
    "Média de Média de Temperatura Máxima (ºC)",
    "Maximo_de_Media_de_Temperatura_Maxima(ºC)",
    "Areia",
    "Limo",
    "Argila",
    "ph"
])

# Nombres de las variables climáticas
variables_clima = resultados_df.columns[2:]

# Función para predecir variables climáticas y del suelo
def predecir_clima(lat, lon):
    coords_escaladas = scaler.transform([[lat, lon]])
    predicciones = modelo.predict(coords_escaladas)
    return {var: round(pred, 2) for var, pred in zip(variables_clima, predicciones[0])}

# Manejador de clic en el mapa
def handle_interaction(**kwargs):
    global current_marker, resultados_df
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')

        # Actualizar marcador
        if current_marker:
            m.remove_layer(current_marker)
        current_marker = Marker(location=(lat, lon))
        m.add_layer(current_marker)

        # Realizar predicción
        predicciones = predecir_clima(lat, lon)

        # Guardar resultados en el DataFrame
        fila = {"Latitud": lat, "Longitud": lon}
        fila.update(predicciones)
        resultados_df = pd.concat([resultados_df, pd.DataFrame([fila])], ignore_index=True)

        # Mostrar resultados
        with coord_output:
            coord_output.clear_output()
            print(f"Coordenadas: ({lat:.6f}, {lon:.6f})")
            print("\nPredicciones climáticas y del suelo:")
            for var, valor in predicciones.items():
                print(f"• {var}: {valor}")

# Botón para mostrar el DataFrame acumulado
def mostrar_dataframe(b):
    with coord_output:
        coord_output.clear_output()
        display(resultados_df)

boton_mostrar_df = Button(description="Mostrar resultados guardados")
boton_mostrar_df.on_click(mostrar_dataframe)

# Registrar eventos del mapa
m.on_interaction(handle_interaction)

# Mostrar componentes
display(VBox([m, boton_mostrar_df, coord_output]))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-5-cbc7e376eacd>:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resultados_df = pd.concat([resultados_df, pd.DataFrame([fila])], ignore_index=True)


In [ ]:
from ipywidgets import ToggleButtons, Button, HBox

# Widgets para indicar si hay riego
toggle_rega = ToggleButtons(
    options=[('Com rega', 1), ('Sem rega', 0)],
    description='Rega:',
    style={'description_width': 'initial'}
)
boton_aplicar_rega = Button(description="Aplicar rega à última entrada")

# Función para aplicar el valor de rega a la última fila del DataFrame (como entero)
def aplicar_rega(b):
    global resultados_df
    if not resultados_df.empty:
        resultados_df.at[resultados_df.index[-1], 'rega'] = int(toggle_rega.value)
        # Asegurar que toda la columna sea del tipo Int64 (soporta nulos)
        resultados_df['rega'] = resultados_df['rega'].astype('Int64')
        with coord_output:
            print(f"Valor de 'rega' ({toggle_rega.label}) aplicado à última entrada.")
    else:
        with coord_output:
            print("O DataFrame ainda está vazio. Adicione uma coordenada primeiro.")

# Asociar función al botón
boton_aplicar_rega.on_click(aplicar_rega)

# Mostrar controles
display(HBox([toggle_rega, boton_aplicar_rega]))



In [ ]:
resultados_df.head()

,Latitud,Longitud,Media_Dias_geada_anual,Media_de_Horas_de_Frio,Media_de_Precipitação_ERA5,Minimo_de_Media_de_Temperatura_Minima(ºC),Média de Média de Temperatura Mínima (ºC),Média de Média de Temperatura Máxima (ºC),Maximo_de_Media_de_Temperatura_Maxima(ºC),Areia,Limo,Argila,ph,rega
0,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1


In [ ]:
# Lista de cultivos a añadir
cultivos = [
    "Abacate_Mexicano", "Abacate_Guatemalteca", "Abacate_Antilhano",
    "Macieira_RoyalGala", "Macieira_Fuji", "Macieira_GrannySmith",
    "Olival_Cobrançosa", "Olival_Arbequina", "Olival_picual",
    "Amendoal_Guara", "Amendoal_Marcona", "Amendoal_Lauranne",
    "Vinha_Tempranilho", "Vinha_Alvarinho", "Vinha_Moscatel",
    "Tomate_EarlyGirl", "Tomate_Sunmaster", "Tomate_pera",
    "Pimento_Padron", "Pimento_Lamuyo", "Pimento_Jalapeño",
    "Pera_Rocha", "Pera_williams", "Pera_Conference",
    "Batata_Desirée", "Batata_Kennebec", "Batata_PurpleMajesty",
    "Milho_mole", "Milho_PioneerP0023", "Milho_DentadoAmarelo"
]

# Crear un nuevo DataFrame expandido
def expandir_por_cultivo(df_original):
    if df_original.empty:
        print("O DataFrame está vazio. Adicione dados antes de expandir por cultivo.")
        return pd.DataFrame()

    # Repetir cada fila len(cultivos) veces
    df_expandido = pd.DataFrame(
        df_original.loc[df_original.index.repeat(len(cultivos))].values,
        columns=df_original.columns
    )

    # Añadir la columna "Cultivos"
    df_expandido["Cultivos"] = cultivos * len(df_original)

    return df_expandido

# Aplicar expansión
resultados_expandido = expandir_por_cultivo(resultados_df)

# Mostrar una vista previa del resultado
with coord_output:
    coord_output.clear_output()
    print("DataFrame expandido por cultivos:")
    display(resultados_expandido.head(15))  # Muestra solo las primeras 15 filas


In [ ]:
resultados_expandido.head(20)

,Latitud,Longitud,Media_Dias_geada_anual,Media_de_Horas_de_Frio,Media_de_Precipitação_ERA5,Minimo_de_Media_de_Temperatura_Minima(ºC),Média de Média de Temperatura Mínima (ºC),Média de Média de Temperatura Máxima (ºC),Maximo_de_Media_de_Temperatura_Maxima(ºC),Areia,Limo,Argila,ph,rega,Cultivos
0,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Abacate_Mexicano
1,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Abacate_Guatemalteca
2,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Abacate_Antilhano
3,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Macieira_RoyalGala
4,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Macieira_Fuji
5,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Macieira_GrannySmith
6,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Olival_Cobrançosa
7,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Olival_Arbequina
8,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Olival_picual
9,37.03764,-7.943115,2.17,188.48,474.27,9.64,15.05,22.19,29.46,44.38,27.58,28.04,7.12,1,Amendoal_Guara


In [ ]:
# Definir rangos ideales para cada cultivo, ahora con soporte para múltiples texturas
cultivos = {
    "Abacate_Mexicano": {
        "dias_geada_max": 3, "horas_frio_min": 0,
        "precipitacion_min": 1000, "precipitacion_max": 1800,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -3, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 5.5, "ph_max": 6.5
    },
    "Abacate_Guatemalteca": {
        "dias_geada_max": 2.2, "horas_frio_min": 0,
        "precipitacion_min": 1200, "precipitacion_max": 2000,
        "temp_minmed_Min": 18, "temp_maxmed_Min": 28,
        "temp_minmin_Min": -2, "temp_maxmax_Min": 37,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 5.5, "ph_max": 7
    },
    "Abacate_Antilhano": {
        "dias_geada_max": 2.2, "horas_frio_min": 0,
        "precipitacion_min": 800, "precipitacion_max": 1600,
        "temp_minmed_Min": 22, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -1, "temp_maxmax_Min": 38,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 6.0, "ph_max": 7.5
    },
    "Macieira_RoyalGala": {
        "dias_geada_max": 5, "horas_frio_min": 555.55,
        "precipitacion_min": 1000, "precipitacion_max": 1200,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -15, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 5.5, "ph_max": 7.0
    },
    "Macieira_Fuji": {
        "dias_geada_max": 5, "horas_frio_min": 550,
        "precipitacion_min": 900, "precipitacion_max": 1100,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -20, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 7.0
    },
    "Macieira_GrannySmith": {
        "dias_geada_max": 5, "horas_frio_min": 600,
        "precipitacion_min": 1100, "precipitacion_max": 1300,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -18, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 7
    },
    "Olival_Cobrançosa": {
        "dias_geada_max": 15, "horas_frio_min": 225,
        "precipitacion_min": 400, "precipitacion_max": 500,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -10, "temp_maxmax_Min": 37,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 6.0, "ph_max": 8.5
    },
    "Olival_Arbequina": {
        "dias_geada_max": 12, "horas_frio_min": 225,
        "precipitacion_min": 400, "precipitacion_max": 500,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -8, "temp_maxmax_Min": 37,
         "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 23, "arena_max": 52, "limo_min": 28, "limo_max": 50, "arcilla_min": 7, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 8.5
    },
    "Olival_picual": {
        "dias_geada_max": 8, "horas_frio_min": 225,
        "precipitacion_min": 500, "precipitacion_max": 700,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -6, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 8.5
    },
    "Amendoal_Guara": {
        "dias_geada_max": 5, "horas_frio_min": 444,
        "precipitacion_min": 350, "precipitacion_max": 600,
        "temp_minmed_Min": 20, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -3, "temp_maxmax_Min": 35,
         "texturas": [
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.5, "ph_max": 8.5
    },
    "Amendoal_Marcona": {
        "dias_geada_max": 2.2, "horas_frio_min": 333,
        "precipitacion_min": 350, "precipitacion_max": 500,
        "temp_minmed_Min": 20, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -2, "temp_maxmax_Min": 35,
         "texturas": [
            {"arena_min": 85, "arena_max": 100, "limo_min": 0, "limo_max": 10, "arcilla_min": 0, "arcilla_max": 10},
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20},
            {"arena_min": 45, "arena_max": 80, "limo_min": 0, "limo_max": 28, "arcilla_min": 20, "arcilla_max": 35},
            {"arena_min": 23, "arena_max": 52, "limo_min": 28, "limo_max": 50, "arcilla_min": 7, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 8.5
    },
    "Amendoal_Lauranne": {
        "dias_geada_max": 10, "horas_frio_min": 666,
        "precipitacion_min": 500, "precipitacion_max": 700,
        "temp_minmed_Min": 20, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -5, "temp_maxmax_Min": 37,
        "texturas": [
            {"arena_min": 85, "arena_max": 100, "limo_min": 0, "limo_max": 10, "arcilla_min": 0, "arcilla_max": 10},
            {"arena_min": 70, "arena_max": 90, "limo_min": 0, "limo_max": 30, "arcilla_min": 0, "arcilla_max": 15},
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20},
            {"arena_min": 45, "arena_max": 80, "limo_min": 0, "limo_max": 28, "arcilla_min": 20, "arcilla_max": 35},
            {"arena_min": 23, "arena_max": 52, "limo_min": 28, "limo_max": 50, "arcilla_min": 7, "arcilla_max": 27}
        ],
        "ph_min": 6.5, "ph_max": 8.5
    },
    "Vinha_Tempranilho": {
        "dias_geada_max": 5, "horas_frio_min": 283,
        "precipitacion_min": 400, "precipitacion_max": 600,
        "temp_minmed_Min": 5, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -8, "temp_maxmax_Min": 35,
       "texturas": [
            {"arena_min": 0, "arena_max": 45, "limo_min": 0, "limo_max": 40, "arcilla_min": 40, "arcilla_max": 100},
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 53, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 0, "arena_max": 20, "limo_min": 40, "limo_max": 73, "arcilla_min": 27, "arcilla_max": 40},
            {"arena_min": 45, "arena_max": 80, "limo_min": 0, "limo_max": 28, "arcilla_min": 20, "arcilla_max": 35},
            {"arena_min": 45, "arena_max": 65, "limo_min": 0, "limo_max": 20, "arcilla_min": 35, "arcilla_max": 55}
        ],
        "ph_min": 6.5, "ph_max": 7.5
    },
    "Vinha_Alvarinho": {
        "dias_geada_max": 4, "horas_frio_min": 283,
        "precipitacion_min": 600, "precipitacion_max": 1200,
        "temp_minmed_Min": 18, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -6, "temp_maxmax_Min": 35,
         "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 5.5, "ph_max": 6.5
    },
    "Vinha_Moscatel": {
        "dias_geada_max": 2.2, "horas_frio_min": 283,
        "precipitacion_min": 500, "precipitacion_max": 700,
        "temp_minmed_Min": -18, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -5, "temp_maxmax_Min": 35,
         "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20},
            {"arena_min": 85, "arena_max": 100, "limo_min": 0, "limo_max": 10, "arcilla_min": 0, "arcilla_max": 10}
        ],
        "ph_min": 6.0, "ph_max": 7.5
    },
    "Tomate_EarlyGirl": {
        "dias_geada_max": 0, "horas_frio_min": 0,
        "precipitacion_min": 600, "precipitacion_max": 900,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": 5, "temp_maxmax_Min": 32,
        "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 6.0, "ph_max": 6.8
    },
    "Tomate_Sunmaster": {
        "dias_geada_max": 3, "horas_frio_min": 0,
        "precipitacion_min": 600, "precipitacion_max": 900,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": 13, "temp_maxmax_Min": 40,
        "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 5.5, "ph_max": 6.5
    },
    "Tomate_pera": {
        "dias_geada_max": 3, "horas_frio_min": 0,
        "precipitacion_min": 600, "precipitacion_max": 900,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": 13, "temp_maxmax_Min": 32,
        "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 6.0, "ph_max": 6.8
    },
    "Pimento_Padron": {
        "dias_geada_max": 1, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 18, "temp_maxmed_Min": 27,
        "temp_minmin_Min": 10, "temp_maxmax_Min": 32,
        "texturas": [
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 5.5, "ph_max": 7.0
    },
    "Pimento_Lamuyo": {
        "dias_geada_max": 1, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 18, "temp_maxmed_Min": 27,
        "temp_minmin_Min": 15, "temp_maxmax_Min": 32,
        "texturas": [
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 5.5, "ph_max": 7.0
    },
    "Pimento_Jalapeño": {
        "dias_geada_max": 1, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 18, "temp_maxmed_Min": 27,
        "temp_minmin_Min": 15, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 5.5, "ph_max": 7.0
    },
    "Pera_Rocha": {
        "dias_geada_max": 8, "horas_frio_min": 200,
        "precipitacion_min": 700, "precipitacion_max": 1200,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -10, "temp_maxmax_Min": 30,
        "texturas": [
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 7.5
    },
    "Pera_williams": {
        "dias_geada_max": 8, "horas_frio_min": 600,
        "precipitacion_min": 600, "precipitacion_max": 800,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -10, "temp_maxmax_Min": 30,
        "texturas": [
           {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 6.0, "ph_max": 7.2
    },
    "Pera_Conference": {
        "dias_geada_max": 10, "horas_frio_min": 700,
        "precipitacion_min": 700, "precipitacion_max": 900,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -15, "temp_maxmax_Min": 30,
        "texturas": [
           {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 6.5, "ph_max": 7.5
    },
    "Batata_Desirée": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 1800,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 25,
        "temp_minmin_Min": -2, "temp_maxmax_Min": 35,
        "texturas": [
             {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 5.5, "ph_max": 6.5
    },
    "Batata_Kennebec": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 10, "temp_maxmed_Min": 20,
        "temp_minmin_Min": -2, "temp_maxmax_Min": 30,
        "texturas": [
             {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 5.5, "ph_max": 6.5
    },
    "Batata_PurpleMajesty": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 10, "temp_maxmed_Min": 22,
        "temp_minmin_Min": -1, "temp_maxmax_Min": 30,
        "texturas": [
             {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20}
        ],
        "ph_min": 5.8, "ph_max": 7.0
    },
    "Milho_mole": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 500, "precipitacion_max": 800,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -1, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 52, "arena_max": 85, "limo_min": 0, "limo_max": 50, "arcilla_min": 0, "arcilla_max": 20},
            {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 5.8, "ph_max": 7.0
    },
    "Milho_PioneerP0023": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 450, "precipitacion_max": 700,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": 0, "temp_maxmax_Min": 35,
        "texturas": [
            {"arena_min": 20, "arena_max": 45, "limo_min": 15, "limo_max": 50, "arcilla_min": 27, "arcilla_max": 40}
        ],
        "ph_min": 6.0, "ph_max": 7.5
    },
    "Milho_DentadoAmarelo": {
        "dias_geada_max": 2, "horas_frio_min": 0,
        "precipitacion_min": 600, "precipitacion_max": 900,
        "temp_minmed_Min": 15, "temp_maxmed_Min": 30,
        "temp_minmin_Min": -2, "temp_maxmax_Min": 32,
        "texturas": [
           {"arena_min": 0, "arena_max": 20, "limo_min": 50, "limo_max": 87, "arcilla_min": 0, "arcilla_max": 27}
        ],
        "ph_min": 5.5, "ph_max": 6.8
    },
}

# Función para evaluar si una variable está dentro del rango óptimo
def evaluar_variable(valor, min_val=None, max_val=None, invertir=False):
    if min_val is not None and max_val is not None:
        if min_val <= valor <= max_val:
            return 1  # Cumple completamente
        elif (min_val - 0.1 * min_val) <= valor <= (max_val + 0.1 * max_val):
            return 0.5  # Cerca del rango ideal
    elif min_val is not None:
        if valor >= min_val:
            return 1
        elif valor >= (min_val * 0.9):
            return 0.5
    elif max_val is not None and invertir:
        if valor <= max_val:
            return 1
        elif valor <= (max_val * 1.1):
            return 0.5
    return 0

# Función para evaluar si alguna textura es válida
def evaluar_textura(arena, limo, arcilla, texturas):
    for textura in texturas:
        cumple_arena = evaluar_variable(arena, textura["arena_min"], textura["arena_max"])
        cumple_limo = evaluar_variable(limo, textura["limo_min"], textura["limo_max"])
        cumple_arcilla = evaluar_variable(arcilla, textura["arcilla_min"], textura["arcilla_max"])
        if all(val > 0 for val in [cumple_arena, cumple_limo, cumple_arcilla]):
            return (cumple_arena + cumple_limo + cumple_arcilla) / 3
    return 0

def calcular_puntuacion_detallada(row):
    cultivo = row["Cultivos"]
    if cultivo not in cultivos:
        return pd.Series({col: 0 for col in [
            'p_dias_geada', 'p_horas_frio', 'p_precipitacion',
            'p_temp_minmed', 'p_temp_maxmed', 'p_temp_minmin',
            'p_temp_maxmax', 'p_textura', 'p_ph', 'puntuacion_final']})

    valores = cultivos[cultivo]
    puntuaciones = {}

    # Evaluación de cada variable con prefijo 'p_' para puntuaciones
    puntuaciones['p_dias_geada'] = evaluar_variable(
        row["Media_Dias_geada_anual"],
        max_val=valores["dias_geada_max"],
        invertir=True
    )

    puntuaciones['p_horas_frio'] = evaluar_variable(
        row["Media_de_Horas_de_Frio"],
        min_val=valores["horas_frio_min"]
    )

    # Manejar regadío (rega) correctamente
    if row["rega"] == 1:
        puntuaciones['p_precipitacion'] = 1.0
    else:
        puntuaciones['p_precipitacion'] = evaluar_variable(
            row["Media_de_Precipitação_ERA5"],
            valores["precipitacion_min"],
            valores["precipitacion_max"]
    )

    puntuaciones['p_temp_minmed'] = evaluar_variable(
        row["Média de Média de Temperatura Mínima (ºC)"],
        min_val=valores["temp_minmed_Min"]
    )

    # Versión corregida para temperaturas máximas
    puntuaciones['p_temp_maxmed'] = evaluar_variable(
        row["Média de Média de Temperatura Máxima (ºC)"],
        max_val=valores["temp_maxmed_Min"],  # Ahora con nombre correcto
        invertir=True
    )

    puntuaciones['p_temp_minmin'] = evaluar_variable(
        row["Minimo_de_Media_de_Temperatura_Minima(ºC)"],
        min_val=valores["temp_minmin_Min"]
    )

    # Versión corregida para temperaturas máximas
    puntuaciones['p_temp_maxmax'] = evaluar_variable(
        row["Maximo_de_Media_de_Temperatura_Maxima(ºC)"],
        max_val=valores["temp_maxmax_Min"],  # Ahora con nombre correcto
        invertir=True

    )


    puntuaciones['p_textura'] = evaluar_textura(
        row["Areia"],
        row["Limo"],
        row["Argila"],
        valores["texturas"]
    )

    puntuaciones['p_ph'] = evaluar_variable(
        row["ph"],
        valores["ph_min"],
        valores["ph_max"]
    )

    # Calcular puntuación final
    puntuaciones['puntuacion_final'] = sum(puntuaciones.values())

    return pd.Series(puntuaciones)

# Aplicar la función corregida
puntuaciones_df = resultados_expandido.apply(calcular_puntuacion_detallada, axis=1)

# Combinar con el dataframe original
resultados_completo = pd.concat([resultados_expandido, puntuaciones_df], axis=1)

# Seleccionar y ordenar columnas
columnas_mostrar = [
    'Cultivos', 'rega', 'ph',
    'p_dias_geada', 'p_horas_frio', 'p_precipitacion',
    'p_temp_minmed', 'p_temp_maxmed', 'p_temp_minmin',
    'p_temp_maxmax', 'p_textura', 'p_ph', 'puntuacion_final'
]

resultados_ordenados = resultados_completo[columnas_mostrar].sort_values(
    by='puntuacion_final',
    ascending=False
).reset_index(drop=True)

# Mostrar resultados
print("Resultados corregidos ordenados por idoneidad:")
display(resultados_ordenados)

Resultados corregidos ordenados por idoneidad:


,Cultivos,rega,ph,p_dias_geada,p_horas_frio,p_precipitacion,p_temp_minmed,p_temp_maxmed,p_temp_minmin,p_temp_maxmax,p_textura,p_ph,puntuacion_final
0,Abacate_Mexicano,1,7.12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,0.5,8.500000
1,Milho_PioneerP0023,1,7.12,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.500000
2,Vinha_Tempranilho,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.000000
3,Olival_picual,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.000000
4,Olival_Arbequina,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.000000
5,Abacate_Antilhano,1,7.12,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.000000,1.0,8.000000
6,Pera_Conference,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.000000
7,Olival_Cobrançosa,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,8.000000
8,Macieira_GrannySmith,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,0.5,7.500000
9,Macieira_Fuji,1,7.12,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.000000,0.5,7.500000
